In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import seaborn as sns
import matplotlib.pyplot as plt

import folium 
from folium import plugins
# from folium.plugins import HeatMap

import json 
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [23]:
patient = pd.read_csv('coronavirusdataset/PatientInfo.csv')
route = pd.read_csv('coronavirusdataset/PatientRoute.csv')
region = pd.read_csv('coronavirusdataset/Region.csv')

In [24]:
patient.head()

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,2.0,male,1964.0,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,1.0,NaN,75.0,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,5.0,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,1.0,NaN,31.0,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,6.0,male,1964.0,50s,Korea,Seoul,Jongno-gu,NaN,contact with patient,2.0,2.002000e+09,17.0,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,7.0,male,1991.0,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,1.0,NaN,9.0,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,9.0,female,1992.0,20s,Korea,Seoul,Seongbuk-gu,NaN,contact with patient,2.0,1.000000e+09,2.0,NaN,2020-01-31,2020-02-24,NaN,released


In [25]:
m = folium.Map([36.5, 128], zoom_start=7) 
# m1 = folium.Map([37, 128], zoom_start=7, tiles='Stamen Terrain') 
# m2 = folium.Map([37, 128], zoom_start=7, tiles='Stamen Toner') 

m

In [14]:
m = folium.Map([36, 128], zoom_start=7) 

plugins.Fullscreen(position='topright',  # Full screen
                   title='Click to Expand', 
                   title_cancel='Click to Exit', 
                   force_separate_button=True).add_to(m)

plugins.MousePosition().add_to(m) ## you can easily get coordinates.

m

In [26]:
regional_patient = pd.merge(patient[['patient_id','confirmed_date','sex','age','province','city']],region[['province','city','latitude','longitude']], how = 'left', on = ['province','city'])
display(regional_patient.head())
print('There are a few missing values')
print('약간의 missing 값들이 있네요. 분석할 때 유의해야할 것 같습니다.')
display(regional_patient.isnull().sum() / regional_patient.shape[0])

,patient_id,confirmed_date,sex,age,province,city,latitude,longitude
0,1000000001,2020-01-23,male,50s,Seoul,Gangseo-gu,37.551166,126.849506
1,1000000002,2020-01-30,male,30s,Seoul,Jungnang-gu,37.606832,127.092656
2,1000000003,2020-01-30,male,50s,Seoul,Jongno-gu,37.572999,126.979189
3,1000000004,2020-01-30,male,20s,Seoul,Mapo-gu,37.566283,126.901644
4,1000000005,2020-01-31,female,20s,Seoul,Seongbuk-gu,37.589562,127.016700


There are a few missing values
약간의 missing 값들이 있네요. 분석할 때 유의해야할 것 같습니다.


patient_id        0.000000
confirmed_date    0.000000
sex               0.030051
age               0.033568
province          0.000000
city              0.024297
latitude          0.037724
longitude         0.037724
dtype: float64

In [18]:
regional_count = regional_patient[['latitude','longitude']].dropna() # drop missing values

m = folium.Map([36, 128], zoom_start=7)

plugins.MousePosition().add_to(m) 

plugins.MarkerCluster(regional_count).add_to(m)

m

In [32]:
heat_data = regional_patient\
.groupby(['latitude','longitude'])['patient_id'].count().reset_index()\
.values\
.tolist()

m = folium.Map([36, 128], zoom_start=7)#, tiles = 'stamentoner')

folium.plugins.HeatMap(heat_data).add_to(m)

m